In [1]:
import tkinter as tk
from tkinter import filedialog
import subprocess
from tkinter import ttk
from table_extraction import TableExtraction
import threading

Using TensorFlow backend.


In [31]:
def update_result_text(message):
    # Вставляем надпись с результатом в метку
    result_label.config(text=message)

def extract_data():
    file_path = filedialog.askopenfilename(filetypes=[("PDF Files", "*.pdf")])
    if not file_path:
        return

    select_button.config(state=tk.DISABLED)  # Блокируем кнопку загрузки файла
    progress_bar.pack(pady=10)  # Показываем прогресс бар перед запуском процесса
    progress_bar.start()  # Запускаем анимацию прогресс-бара

    def extract_in_thread():
        try:
            extractor = TableExtraction()
            extractor.extract_from_file(file_path)
            root.after(2000, update_result_text, "Data successfully extracted and processed.")
        except subprocess.CalledProcessError:
            root.after(1000, update_result_text, "Error executing the program")
        finally:
            root.after(1000, enable_button)  # Включаем кнопку загрузки файла после завершения операции
            progress_bar.pack_forget()  # Скрываем прогресс бар после завершения процесса
            progress_bar.stop()  # Останавливаем анимацию прогресс-бара

    # Запускаем извлечение данных в отдельном потоке
    thread = threading.Thread(target=extract_in_thread)
    thread.start()

def enable_button():
    select_button.config(state=tk.NORMAL)  # Включаем кнопку загрузки файла
    progress_bar.stop()  # Останавливаем анимацию прогресс-бара

# Создаем графический интерфейс
root = tk.Tk()
root.title("Extract data from PDF tables")

# Increase the font size
font_size = 16

# Set the window size to a quarter of the screen resolution
screen_width = root.winfo_screenwidth()
screen_height = root.winfo_screenheight()
window_width = screen_width // 2
window_height = screen_height // 2
root.geometry(f"{window_width}x{window_height}")

# Set dark gray background
root.configure(bg="gray15")

# Progress bar
progress_bar = ttk.Progressbar(root, orient=tk.HORIZONTAL, mode='indeterminate', maximum=100, value=0)
progress_bar.pack(pady=10)
progress_bar.pack_forget()

# Button to select the PDF file
select_button = tk.Button(root, text="Выбрать PDF файл", font=("Helvetica", font_size), fg="white", bg="gray35", activebackground="gray35", command=extract_data)
select_button.pack(pady=10)

# Center the button in the window
select_button.place(relx=0.5, rely=0.5, anchor=tk.CENTER)

# Label for displaying the result message
result_label = tk.Label(root, text="", font=("Helvetica", font_size), fg="white", bg="gray15")
result_label.pack(pady=5)

# Run the graphical user interface
root.mainloop()